In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from tkinter import  *
from tkinter.filedialog import askopenfilename
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd
from sklearn.neighbors import DistanceMetric
from math import radians
import pandas as pd 
import numpy as np

root = Tk(className= "Optimal Vehical Route")
root.configure(bg='grey')
root.geometry('995x552') 

def import_csv_data():
    global csv_file_path
    csv_file_path = askopenfilename()
    v.set(csv_file_path)
    txt.insert(1.0 ,main())

    

def load_data():
    data = {}
    df = pd.read_excel(csv_file_path)
    #converting Latitude and Logitude degree data to radians
    df['Latitude'] = np.radians(df['Latitude'])
    df['Longitude'] = np.radians(df['Longitude'])
    # using built in function to find distance in KM unit by Haversine formula
    dist = DistanceMetric.get_metric('haversine')
    df1 =  dist.pairwise(df[['Latitude','Longitude']].to_numpy())*6373
    data['distance_matrix'] = df1
    data['num_vehicles'] = 3
    data['depot'] = 122
    return data

def packaged_function(data):
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']), data['num_vehicles'], data['depot'])

    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()

    search_parameters.local_search_metaheuristic = (
      routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)

    search_parameters.time_limit.seconds = 90
    search_parameters.log_search = True

    # Solve the problem.
    assignment = routing.SolveWithParameters(search_parameters)

    return (manager, routing, assignment)


def print_solution(data,manager, routing, assignment):
    """Prints solution on console."""
    max_route_distance = 0
    a  = ""
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = assignment.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        
        a = a +"\n"+ plan_output
        max_route_distance = max(route_distance, max_route_distance)
    b =   ('Maximum of the route distances: {}m'.format(max_route_distance))
    final = a +"\n"+  "\n" + b
    return final



    
    
def main():
    data = load_data()
    (manager, routing, assignment) = packaged_function(data)
    return print_solution(data,manager, routing, assignment)

#create canvas
my_canvas = Canvas(root,width = 995, height = 552)
my_canvas.pack(fill = "both", expand = True)

# Creating and set background image
background_image = PhotoImage(file ='World.png')
background_label = Label(root, image=background_image)
background_label.place(relwidth=1, relheight=1)

# creating label
label = Label(root, relief =RAISED , font=("calibri bold", 18),padx = 5, pady = 5, text='File Path')
label.place(x=250, y= 60)

# creating entry for input file path
v = StringVar()
entry = Entry(root,font=("calibri bold", 20),width = 27, textvariable=v)
entry.place(x=360, y= 60)

# creating button to search for file
button = Button(root,relief = RAISED, font=("calibri bold", 18),text='Browse Data Set',pady = 5,command=import_csv_data)
button.place(x=360, y=470)

# creating text for displaying optimal route
txt = Text(root, width = 50, height = 25, wrap = WORD)
txt.place(x=360, y=120)


root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/hemal/opt/anaconda3/lib/python3.8/tkinter/__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-1-e9aab1aa0aa1>", line 22, in import_csv_data
    txt.insert(1.0 ,main())
  File "<ipython-input-1-e9aab1aa0aa1>", line 114, in main
    data = load_data()
  File "<ipython-input-1-e9aab1aa0aa1>", line 28, in load_data
    df = pd.read_excel(csv_file_path)
  File "/Users/hemal/opt/anaconda3/lib/python3.8/site-packages/pandas/util/_decorators.py", line 296, in wrapper
    return func(*args, **kwargs)
  File "/Users/hemal/opt/anaconda3/lib/python3.8/site-packages/pandas/io/excel/_base.py", line 304, in read_excel
    io = ExcelFile(io, engine=engine)
  File "/Users/hemal/opt/anaconda3/lib/python3.8/site-packages/pandas/io/excel/_base.py", line 867, in __init__
    self._reader = self._engines[engine](self._io)
  File "/Users/hemal/opt/anaconda3/lib/python3.8/site-packages/pandas/io